In [2]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

## 1. Basic autograd example 1

In [3]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

In [4]:
# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

In [5]:
# Compute gradients.
y.backward()

In [6]:
# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2.)
tensor(1.)
tensor(1.)


## 2. Basic autograd example 2

In [7]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

In [37]:
# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight, linear.weight.size())
print ('b: ', linear.bias, linear.bias.size())

w:  Parameter containing:
tensor([[ 0.5612, -0.1791,  0.5277],
        [ 0.2744, -0.3818, -0.5036]], requires_grad=True) torch.Size([2, 3])
b:  Parameter containing:
tensor([ 0.0368, -0.0265], requires_grad=True) torch.Size([2])


In [38]:
# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [39]:
# Forward pass.
pred = linear(x)

In [40]:
# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

loss:  1.4988620281219482


In [41]:
# Backward pass.
loss.backward()

In [42]:
# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

dL/dw:  tensor([[ 0.3951,  0.0939,  0.1697],
        [-0.3980, -0.4435, -0.4744]])
dL/db:  tensor([ 0.2572, -0.1830])


In [43]:
# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

In [44]:
# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss after 1 step optimization:  1.4902105331420898


## 3. Loading data from numpy

In [45]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

In [47]:
# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

In [48]:
# Convert the torch tensor to a numpy array.
z = y.numpy()

## 4. Input pipline 

In [49]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

In [50]:
# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

torch.Size([3, 32, 32])
6


In [51]:
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [52]:
# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

In [53]:
# Mini-batch images and labels.
images, labels = data_iter.next()

In [54]:
# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

## 5. Input pipline for custom dataset  

In [56]:
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

In [ ]:
# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)

## 6. Pretrained model

In [58]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

In [59]:
# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

In [60]:
# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

In [61]:
# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

torch.Size([64, 100])


## 7. Save and load the model

In [ ]:
# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

In [ ]:
# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))